Connect to vCenter Server
===

- Connect to vCenter
- Load the content

In [3]:
import config
import vsphere_client
from pyVmomi import vim

# vc-39
# vc-169
conf = config.load_yml_conf(
    '/Users/weijin.wang/ws/.secrets/appflows.yml', 'vc-169')

vcc = vsphere_client.create_vsphere_connection(
    hostname=conf['url'],
    username=conf['username'],
    password=conf['password']
)
content = vsphere_client.load_vsphere_contents(vcc)
print('Loaded contents')

_vms = vsphere_client.list_objects_by_type(content, [vim.VirtualMachine])
print('Find {} vms'.format(len(_vms)))

_datastores = vsphere_client.list_objects_by_type(content, [vim.Datastore])
print('Find {} datastores'.format(len(_datastores)))
_accessible_datastores = [ds for ds in _datastores if ds.summary.accessible]
print('\t{} of them are accessible'.format(len(_accessible_datastores)))

_hosts = vsphere_client.list_objects_by_type(content, [vim.ComputeResource])
print('Find {} hosts:'.format(len(_hosts)))
for host in _hosts:
    print('\t{}'.format(host.name))



Loaded contents
Find 85 vms
Find 138 datastores


	9 of them are accessible
Find 1 hosts:
	esx-7-148.colo.rubrik.com


In [4]:

vms = vsphere_client.list_objects_by_type(content, [vim.VirtualMachine])
print('find {} vms'.format(len(vms)))
for vm in vms:
    print(vm.name)


find 85 vms
Rubrik_Managed_autotest-prem--20200310-114022-121564 03-10 11:54
Rubrik_Managed_autotest-prem--20200310-111910-418623 03-10 11:45
Rubrik_Managed_autotest-prem--20200313-225232-336744 03-13 23:04
Rubrik_Managed_autotest-prem--20200312-155133-771679 03-12 16:05
Rubrik_Managed_autotest-prem--20200312-121555-219005 03-12 12:25
april-07
Rubrik_Managed_april-06 02-28 14:49
april-static-03


Centos6-7-2disk
april-win-01
april-onprem-10
Deprecated demo-vm-03 02-29 04:05
autotest-prem--20200310-111910-418623-test 03-13 17:10
Deprecated april-06 02-28 22:28
Deprecated april-03 02-27 23:11
rohit-centos-6-10-2disk
autotest-prem--20200305-173708-385643
april-static-04
Rubrik_Managed_april-onprem-02 02-14 14:32 02-14 16:52


april-onprem-04
april-onprem-09
april-onprem-06
april-cent7-nocloudinit
Deprecated april-onprem-02 02-27 22:59
hilary-2-test-vm
win_server_2012r2_20gb_4mem_1cpu
rohit-rhel-6-10-2disks


april-03
Deprecated demo-vm-05 03-02 19:44
Rubrik_Managed_april-onprem-02 02-20 11:51 02-22 21:03
Centos-7-0-template
demo-vm-01
april-onprem-02
april-static-01
april-onprem-05
demo-vm-03
rohit-rhel-6-9
Deprecated april-04 03-02 23:25
metadata_ubuntu_vm1


ubuntu_base_template
demo-vm-04
template-centos7-onpremscript
april-onprem-01
demo-vm-02
april-static-02
centos7-onprem-template
ubuntu_4gb_template
april-onprem-11
autotest-prem--20200312-190041-728495
april-06-test 03-04 11:07
april-08-test 03-11 03:11


Rubrik_Managed_autotest-prem--20200306-121741-816320 03-07 15:30
Rubrik_Managed_april-05 02-27 15:47
autotest-prem--20200305-113701-524545
Rubrik_Managed_koki-static-test 03-03 16:41
april-02
Rubrik_Managed_april-08 03-10 17:22
CdmToCdmRohitCentos6_10
Rubrik_Managed_april-06 03-09 23:50
test export april prod05
april-onprem-08
curious george


Rubrik_Managed_april-05 03-09 23:49
Rubrik_Managed_april-onprem-03 02-14 14:22 02-14 14:55
Deprecated april-04 03-03 01:08
Rubrik_Managed_autotest-prem--20200305-113701-524545 03-05 19:45
_weijin_window_vm
april-onprem-07
centos7-prem-template
Deprecated _weijin_0301 03-03 21:38
centos7-tool-template


Rubrik_Managed_april-static-01 03-07 19:44
Rubrik_Managed_april-05 02-27 16:07
Deprecated _weijin_03 02-23 05:36
CdmToCdmRohitRhel6_7
Rubrik_Managed_april-05 02-27 15:47*
Deprecated demo-vm-04 02-29 04:05


Deprecated demo-vm-06 03-02 19:44
centos7-postscript-template
rohit_rhel_6_7
april-onprem-03
april-01
Deprecated april-01 02-27 22:30
hilary-test-vm
curious george*


In [16]:

vm = vsphere_client.get_obj_by_name(
    content, [vim.VirtualMachine], 'april-05')

template = vm
if template:
    print('Template found: {}'.format(template.name))
else:
    print("No template found")


Template found: april-05


In [19]:
for i in range(5):
    vsphere_client.clone_vm(content, template, "_weijin_0310_{}".format(i+2))

Cloning VM to ...
	 Data Center  : 'blueprint'
	 VM Folder    : 'vm'
	 Resource Pool: 'Resources'
	 Data store   : 'datastore1'


VM '_weijin_0310_2' cloned!


Cloning VM to ...
	 Data Center  : 'blueprint'
	 VM Folder    : 'vm'
	 Resource Pool: 'Resources'
	 Data store   : 'datastore1'


VM '_weijin_0310_3' cloned!


Cloning VM to ...
	 Data Center  : 'blueprint'
	 VM Folder    : 'vm'
	 Resource Pool: 'Resources'
	 Data store   : 'datastore1'


...

Cloning VM to ...
	 Data Center  : 'blueprint'
	 VM Folder    : 'vm'
	 Resource Pool: 'Resources'
	 Data store   : 'datastore1'


VM '_weijin_0310_5' cloned!


Cloning VM to ...
	 Data Center  : 'blueprint'


	 VM Folder    : 'vm'
	 Resource Pool: 'Resources'
	 Data store   : 'datastore1'


VM '_weijin_0310_6' cloned!


In [22]:
vms = vsphere_client.filter_vms_by_name(content, "_weijin")
for vm in vms:
    print(vm.name)


_weijin_0310_6
_weijin_0310_5
_weijin_0310_4
_weijin_0310_2
_weijin_0310_1
_weijin_0310_3


In [9]:
for vm in vms:
    print("Powering off VM: %s" % vm.name)
    task = vm.PowerOffVM_Task()
    vsphere_client.wait_for_task(task)
    print("task status: {}".format(task.info.state))
        
    print("Destroying VM: %s" % vm.name)
    task = vm.Destroy_Task()
    vsphere_client.wait_for_task(task)
    print("task status: {}".format(task.info.state))



_weijin_0310_6
_weijin_0310_5
_weijin_0310_4
_weijin_0310_2
_weijin_0310_1
_weijin_0310_3
